## Simple Threat Analysis with Pandas

This notebook demonstrates a simple way to identify known bad actors within egress transaction generated from wire data traffic. Python Pandas is used to compare the IP addresses of servers exfiltrating from our 'friendly" set of IP addresses against known threats. 
 
This example uses IP addresses that have been reported within the last 48 hours as having run attacks on the service Mail, Postfix at the Blocklist website: https://lists.blocklist.de/lists/mail.txt, downloaded Jan 9

In [35]:
import pandas as pd
import os
import numpy

This test 'friendly' data is 10,000 records of egress transaction.
Read the data in pandas. 

In [129]:
egress_test=pd.read_csv("Traffic.csv", encoding='latin-1')

In [139]:
egress.head(10)

,Time,Record Type,Source,Destination,Source Location,Dest Location,Environment,Dest Country,Protocol,Client Address,Client Bytes,Server Address,Server Bytes,Latency,Process Time
0,2018-01-08 22:55:39.440,Flow Audit,External,External,DC Datacenter,DC Datacenter,EGRESS,United States,telnet,172.45.8.6,3,172.45.8.56,36,NaN,NaN
1,2018-01-08 22:55:38.524,Flow Audit,External,External,DC Datacenter,DC Datacenter,EGRESS,United States,tcp:23,172.45.8.9,1,172.45.8.59,1,NaN,17.626
2,2018-01-08 22:55:38.516,Flow Audit,Santa Clara Campus (Users),External,Santa Clara Office,California,EGRESS,United States,SSL:443,192.168.0.104,173,173.194.33.144,133,NaN,NaN
3,2018-01-08 22:55:38.516,Flow Audit,Santa Clara Campus (Users),External,Santa Clara Office,California,EGRESS,United States,SSL:443,192.168.0.104,600,173.194.33.144,"1,406",NaN,0.747
4,2018-01-08 22:55:38.282,Flow Audit,External,External,DC Datacenter,DC Datacenter,EGRESS,United States,tcp:23,172.45.8.9,1,172.45.8.59,1,NaN,64.652
5,2018-01-08 22:55:38.033,Flow Audit,External,External,DC Datacenter,DC Datacenter,EGRESS,United States,tcp:23,172.45.8.9,3,172.45.8.59,3,NaN,"548,711.657"
6,2018-01-08 22:55:36.551,Flow Audit,Santa Clara Campus (Users),External,Santa Clara Office,Massachusetts,EGRESS,United States,HTTP,192.168.0.105,231,23.5.251.27,"1,755",NaN,20.179
7,2018-01-08 22:55:36.551,Flow Audit,Santa Clara Campus (Users),External,Santa Clara Office,Massachusetts,EGRESS,United States,HTTP,192.168.0.105,"8,011",23.5.245.163,"11,680",20.052,21.745
8,2018-01-08 22:55:36.551,Flow Audit,Santa Clara Campus (Users),External,Santa Clara Office,Oregon,EGRESS,United States,SSL:443,192.168.0.105,229,140.211.11.131,"7,583",NaN,27.768
9,2018-01-08 22:55:36.516,Flow Audit,Santa Clara Campus (Users),External,Santa Clara Office,California,EGRESS,United States,SSL:443,192.168.0.99,209,151.101.48.124,"71,457",NaN,48.037


We can quickly identify the countries and volume of traffice associated with the egress servers and dig deeper into transactions linked to a country of concern, for example 'France.

In [39]:
servers=egress['Dest Country']

In [40]:
servers.value_counts()

United States     9580
Norway             291
Ireland             51
Japan               22
France              13
Netherlands         10
Singapore            5
United Kingdom       1
Poland               1
Name: Dest Country, dtype: int64

In [134]:
France=(egress.loc[egress['Dest Country'].isin(['France'])])

In [138]:
France.head()

,Time,Record Type,Source,Destination,Source Location,Dest Location,Environment,Dest Country,Protocol,Client Address,Client Bytes,Server Address,Server Bytes,Latency,Process Time
1784,2018-01-08 22:50:13.385,Flow Audit,Santa Clara Campus (Users),External,Santa Clara Office,Ãle-de-France,EGRESS,France,SSL:443,192.168.0.99,126,74.121.138.36,258,NaN,57.071
1787,2018-01-08 22:50:13.327,Flow Audit,Santa Clara Campus (Users),External,Santa Clara Office,Ãle-de-France,EGRESS,France,SSL:443,192.168.0.99,517,74.121.138.36,"2,916",60.99,76.425
3179,2018-01-08 22:45:26.698,Flow Audit,Santa Clara Campus (Users),External,Santa Clara Office,NaN,EGRESS,France,SSL:443,192.168.0.99,51,188.165.39.118,216,NaN,"11,914.278"
3180,2018-01-08 22:45:26.691,Flow Audit,Santa Clara Campus (Users),External,Santa Clara Office,NaN,EGRESS,France,SSL:443,192.168.0.99,51,188.165.39.118,216,NaN,"11,915.278"
3317,2018-01-08 22:45:11.081,Flow Audit,Santa Clara Campus (Users),External,Santa Clara Office,Ãle-de-France,EGRESS,France,SSL:443,192.168.0.99,517,5.135.135.51,137,134.207,133.578


We can see that there are 13 transactions, all from the Santa Clara Campus, the most frequent destinationis Ale-de-France and 188.165.39.118 is the most common server address.	

In [137]:
France.describe()

,Time,Record Type,Source,Destination,Source Location,Dest Location,Environment,Dest Country,Protocol,Client Address,Client Bytes,Server Address,Server Bytes,Latency,Process Time
count,13,13,13,13,13,5,13,13,13,13,13,13,13,6,12
unique,12,1,1,1,1,1,1,1,1,1,8,4,9,6,12
top,2018-01-08 22:45:11.081,Flow Audit,Santa Clara Campus (Users),External,Santa Clara Office,Ãle-de-France,EGRESS,France,SSL:443,192.168.0.99,517,188.165.39.118,137,138.476,139.197
freq,2,13,13,13,13,5,13,13,13,13,3,4,2,1,1


In [41]:
server_ip=egress['Server Address']

We can also determen the frequency of each egree IP address in the test data set.

In [42]:
server_ip.describe()

count            10000
unique             691
top       52.70.108.65
freq               225
Name: Server Address, dtype: object

In [43]:
server_ip.value_counts()

52.70.108.65       225
152.195.32.119     221
208.43.234.47      196
74.125.30.189      181
173.194.33.75      152
192.229.163.180    147
69.172.216.111     145
104.96.86.223      142
108.174.11.65      132
192.0.72.2         114
173.194.33.69      105
23.59.189.82       104
172.45.8.97        101
91.203.99.19        99
172.45.8.81         97
172.45.8.83         94
172.45.8.98         92
172.45.8.62         90
172.45.8.86         90
172.45.8.78         89
173.194.33.95       88
172.45.8.74         86
172.45.8.72         84
172.45.8.93         84
172.45.8.58         82
172.45.8.52         81
172.45.8.96         80
172.45.8.55         80
172.45.8.67         78
74.125.28.106       78
                  ... 
172.45.7.70          1
172.45.7.81          1
172.45.6.98          1
172.217.9.170        1
172.45.6.92          1
193.221.113.53       1
172.45.6.94          1
31.13.67.16          1
74.6.137.78          1
151.101.56.124       1
52.200.167.6         1
172.45.6.96          1
172.45.6.80

For our purposes we will extract just the egress 'Server Address' column.

In [44]:
justip=egress['Server Address']

In [45]:
justip=justip.to_frame()

In [46]:
type(justip)

pandas.core.frame.DataFrame

In [55]:
justip.columns =['Server Address']


In [80]:
justip.tail(10)

,Server Address
9990,173.194.33.75
9991,173.194.33.75
9992,173.194.33.75
9993,173.194.33.75
9994,173.194.33.75
9995,173.194.33.65
9996,173.194.33.94
9997,54.230.69.112
9998,54.230.69.112
9999,54.230.69.112


We now have a list of IP addresses for our egress transactions that we want to compare against a list of known 'badactor' sites, called 'blocklist'. Blocklist contains 20,135 addreses. 

In [57]:
blocklist = pd.read_csv('badips.csv', header=None, names=['Server Address'])

In [114]:
blocklist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20135 entries, 0 to 20134
Data columns (total 1 columns):
Server Address    20135 non-null object
dtypes: object(1)
memory usage: 157.4+ KB


In [81]:
blocklist.tail(10)

,Server Address
20125,96.47.224.42
20126,96.44.142.250
20127,195.254.134.10
20128,195.254.134.194
20129,188.95.234.6
20130,188.143.235.21
20131,95.134.130.182
20132,95.143.192.159
20133,176.100.75.27
20134,176.221.42.32


In [ ]:
We merge the two data sets and find that there are 0 overlapping records.
Congratulations! These transactions are not terminating at a known malicious site.

In [102]:
joined_ips=pd.merge(egress,blocklist, on='Server Address', how='inner')

In [103]:
joined_ips.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 15 columns):
Time               0 non-null object
Record Type        0 non-null object
Source             0 non-null object
Destination        0 non-null object
Source Location    0 non-null object
Dest Location      0 non-null object
Environment        0 non-null object
Dest Country       0 non-null object
Protocol           0 non-null object
Client Address     0 non-null object
Client Bytes       0 non-null object
Server Address     0 non-null object
Server Bytes       0 non-null object
Latency            0 non-null object
Process Time       0 non-null object
dtypes: object(15)
memory usage: 0.0+ bytes


In [104]:
joined_ips.tail(20)

,Time,Record Type,Source,Destination,Source Location,Dest Location,Environment,Dest Country,Protocol,Client Address,Client Bytes,Server Address,Server Bytes,Latency,Process Time


Lets add some fake data into our blocklist to make sure the join in working.  I have taken 10 IP addresses from the 'egress' data frame and concatonated it to the original blocklist to create fakeblocklist.

In [141]:
fakeblocklist=pd.read_csv('fakebadservers.csv', names=['Server Address'])

In [145]:
fakeblocklist=pd.concat([blocklist, fakeblocked], axis=0)

In [146]:
fakeblocklist.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20145 entries, 0 to 9
Data columns (total 1 columns):
Server Address    20145 non-null object
dtypes: object(1)
memory usage: 314.8+ KB


In [148]:
fakeblocklist.head(3)

,Server Address
0,1.173.37.30
1,1.192.229.40
2,1.194.116.78


Merge egress and fakeblock list identifies 580 transactions linked to a known 'bad actor'. 

In [116]:
joined_ips2=pd.merge(egress,fakeblocklist, on='Server Address', how='inner')

In [117]:
joined_ips2.head(10)

,Time,Record Type,Source,Destination,Source Location,Dest Location,Environment,Dest Country,Protocol,Client Address,Client Bytes,Server Address,Server Bytes,Latency,Process Time
0,2018-01-08 22:55:38.524,Flow Audit,External,External,DC Datacenter,DC Datacenter,EGRESS,United States,tcp:23,172.45.8.9,1,172.45.8.59,1,NaN,17.626
1,2018-01-08 22:55:38.282,Flow Audit,External,External,DC Datacenter,DC Datacenter,EGRESS,United States,tcp:23,172.45.8.9,1,172.45.8.59,1,NaN,64.652
2,2018-01-08 22:55:38.033,Flow Audit,External,External,DC Datacenter,DC Datacenter,EGRESS,United States,tcp:23,172.45.8.9,3,172.45.8.59,3,NaN,"548,711.657"
3,2018-01-08 22:55:36.461,Flow Audit,External,External,DC Datacenter,DC Datacenter,EGRESS,United States,tcp:23,172.45.8.9,1,172.45.8.59,1,NaN,297.186
4,2018-01-08 22:55:36.164,Flow Audit,External,External,DC Datacenter,DC Datacenter,EGRESS,United States,tcp:23,172.45.8.9,1,172.45.8.59,4,NaN,61.68
5,2018-01-08 22:55:35.265,Flow Audit,External,External,DC Datacenter,DC Datacenter,EGRESS,United States,tcp:23,172.45.8.9,1,172.45.8.59,4,NaN,35.814
6,2018-01-08 22:55:34.982,Flow Audit,External,External,DC Datacenter,DC Datacenter,EGRESS,United States,tcp:23,172.45.8.9,1,172.45.8.59,4,NaN,1.891
7,2018-01-08 22:55:34.847,Flow Audit,External,External,DC Datacenter,DC Datacenter,EGRESS,United States,tcp:23,172.45.8.9,1,172.45.8.59,4,NaN,25.646
8,2018-01-08 22:55:34.685,Flow Audit,External,External,DC Datacenter,DC Datacenter,EGRESS,United States,tcp:23,172.45.8.9,3,172.45.8.59,3,NaN,91.937
9,2018-01-08 22:55:32.952,Flow Audit,External,External,DC Datacenter,DC Datacenter,EGRESS,United States,tcp:23,172.45.8.9,1,172.45.8.59,1,NaN,4.178


In [128]:
joined_ips2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 580 entries, 0 to 579
Data columns (total 15 columns):
Time               580 non-null object
Record Type        580 non-null object
Source             580 non-null object
Destination        580 non-null object
Source Location    580 non-null object
Dest Location      580 non-null object
Environment        580 non-null object
Dest Country       580 non-null object
Protocol           580 non-null object
Client Address     580 non-null object
Client Bytes       573 non-null object
Server Address     580 non-null object
Server Bytes       573 non-null object
Latency            193 non-null object
Process Time       539 non-null object
dtypes: object(15)
memory usage: 72.5+ KB


In [154]:
joined_ips2.head(3)

,Time,Record Type,Source,Destination,Source Location,Dest Location,Environment,Dest Country,Protocol,Client Address,Client Bytes,Server Address,Server Bytes,Latency,Process Time
0,2018-01-08 22:55:38.524,Flow Audit,External,External,DC Datacenter,DC Datacenter,EGRESS,United States,tcp:23,172.45.8.9,1,172.45.8.59,1,NaN,17.626
1,2018-01-08 22:55:38.282,Flow Audit,External,External,DC Datacenter,DC Datacenter,EGRESS,United States,tcp:23,172.45.8.9,1,172.45.8.59,1,NaN,64.652
2,2018-01-08 22:55:38.033,Flow Audit,External,External,DC Datacenter,DC Datacenter,EGRESS,United States,tcp:23,172.45.8.9,3,172.45.8.59,3,NaN,"548,711.657"
